<a href="https://colab.research.google.com/github/jangjung-coding/lettuce_predict_ai/blob/main/lettuce_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Jupyter note상에 그래프 결과 포함 매직 명령
%matplotlib inline

# Matplotlib 기본 글꼴 정의
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

import sys

# 노트북이 코랩에서 실행 중인지 체크합니다.
if 'google.colab' in sys.modules:
    !echo 'debconf debconf/frontend select Noninteractive' | debconf-set-selections

    # 나눔 폰트를 설치합니다.
    !sudo apt-get -qq -y install fonts-nanum
    import matplotlib.font_manager as fm
    font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
    for fpath in font_files:
        fm.fontManager.addfont(fpath)

#     # 나눔바른고딕 폰트로 설정합니다.
    import matplotlib.pyplot as plt
    plt.rc('font', family='NanumBarunGothic')

#     # 마이너스 기호 표시 오류 수정
    import matplotlib
    matplotlib.rcParams['axes.unicode_minus'] = False

# 경고 문제 출력 제한
import warnings

warnings.filterwarnings( 'ignore' )

In [248]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import glob
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_validate,train_test_split, KFold, TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import xgboost
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor

In [136]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [137]:
%cd /content/drive/MyDrive/googleCloud AI 인재양성 교육/[프로젝트]상추의 생육 환경 예측 AI

/content/drive/MyDrive/googleCloud AI 인재양성 교육/[프로젝트]상추의 생육 환경 예측 AI


In [138]:
# 전체 데이터 경로
all_input_list = sorted(glob.glob('./data/train_input/*.csv'))
all_target_list = sorted(glob.glob('./data/train_target/*.csv'))

test_input_list = sorted(glob.glob('./data/test_input/*.csv'))
test_target_list = sorted(glob.glob('./data/test_target/*.csv'))

# EDA

# Preprocessing

In [251]:
# 데이터 전처리
def merge_data(input_list,target_list,col_list,drop_feature_list):
  data_list = []
  label_list = []

  for input_path, target_path in tqdm(zip(input_list,target_list)):
    input_df = pd.read_csv(input_path)
    target_df = pd.read_csv(target_path)

    # 날짜 컬럼 전처리
    if 'obs_time' in input_df.columns:
      input_df['obs_time'] = input_df['obs_time'].map(lambda x: int(x[:2])) # obs_time 컬럼이 있는 경우, 이를 시간으로 변환합니다 (예: "12:00" -> 12)

    # 시간별 파생변수 생성(온도,습도,이산화탄소 반영)
    # input_df['습도온도'] = input_df['내부습도관측치'] * input_df['내부온도관측치']
    # input_df['습도이산화탄소'] = input_df['내부습도관측치'] * input_df['co2관측치']
    # input_df['온도이산화탄소'] = input_df['내부온도관측치'] * input_df['co2관측치']
    # input_df['습도온도이산화탄소'] = input_df['내부습도관측치'] * input_df['내부온도관측치'] * input_df['co2관측치']
    # input_df['광량여부'] = (input_df['시간당백색광량'] + input_df['시간당적색광량'] + input_df['시간당청색광량']).map(lambda x: 0 if x == 0 else 1)

    # 변수 선택
    if len(drop_feature_list) > 0:
      input_df = input_df.drop(columns=drop_feature_list)
      input_df = input_df.fillna(0)
    else:
      input_df = input_df.fillna(0)

    input_length = int(len(input_df)/24)
    target_length = int(len(target_df))

    for idx in range(target_length):
        # 시간별 데이터 하루 단위로 자르기
        day_data = input_df[24*idx:24*(idx+1)]

        # 하루 단위 데이터 flatten
        time_series = day_data.values.flatten().tolist() + [idx]
        data_list.append(time_series)

    for label in target_df["predicted_weight_g"]:
        label_list.append(label)

  # 컬럼명 리스트
  feature_list = [x for x in col_list if x not in drop_feature_list]
  total_col = []
  for hour in range(24):
    for col in feature_list:
      total_col.append(f'{hour}시_{col}')

  total_col.append('day')

  input = pd.DataFrame(data_list,columns=total_col)
  target = pd.DataFrame(label_list)

  # 이상값 수정
  input.loc[input['15시_내부온도관측치'] == 0, ['15시_내부온도관측치','15시_내부습도관측치']] = 27.46201254701998,51.560314940325554
  input.loc[input['1시_시간당분무량'] < 0,'1시_시간당분무량'] = 0
  input.loc[input['15시_시간당백색광량'] < 0 ,['15시_시간당백색광량','15시_시간당적색광량','15시_시간당청색광량']] = 18255.1900 , 1813.6608 , 3565.3540
  return input, target

In [252]:
# target을 성장치로 수정(0일차의 target은 0일차의 상추 중량)
def modified_target(target_df):
  zero_start = []
  for i in range(0,len(target_df)):
    if i%28 == 0:
      zero_start.append(target_df[0][i])

    else:
      num = target_df[0][i] - target_df[0][i-1]
      zero_start.append(num)

  target_df[0] = zero_start
  return target_df

In [253]:
# 총 변수 리스트
# col_list = ['DAT','obs_time','내부온도관측치', '내부습도관측치', 'co2관측치','ec관측치','시간당분무량',
#        '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
#        '일간누적청색광량', '시간당총광량', '일간누적총광량','습도온도','습도이산화탄소','온도이산화탄소','습도온도이산화탄소','광량여부']
col_list = ['DAT','obs_time','내부온도관측치', '내부습도관측치', 'co2관측치','ec관측치','시간당분무량',
       '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
       '일간누적청색광량', '시간당총광량', '일간누적총광량']

# 제거할 변수 선택
drop_feature_list = ['DAT','obs_time','일간누적분무량','일간누적백색광량', '일간누적적색광량' ,'일간누적청색광량', '일간누적총광량','시간당총광량']

# 사용할 변수 리스트
feature_list = [x for x in col_list if x not in drop_feature_list]

# 데이터 합치고 파생변수 생성
train_X, train_y = merge_data(all_input_list,all_target_list,col_list,drop_feature_list)
test_X, test_y = merge_data(test_input_list,test_target_list,col_list,drop_feature_list)

predict_feature_list = train_X.columns
print(feature_list)
print(train_X.shape,len(train_y))
print(test_X.shape,len(test_y))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

['내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량']
(784, 193) 784
(140, 193) 140


In [254]:
train_X.head()

,0시_내부온도관측치,0시_내부습도관측치,0시_co2관측치,0시_ec관측치,0시_시간당분무량,0시_시간당백색광량,0시_시간당적색광량,0시_시간당청색광량,1시_내부온도관측치,1시_내부습도관측치,...,22시_시간당청색광량,23시_내부온도관측치,23시_내부습도관측치,23시_co2관측치,23시_ec관측치,23시_시간당분무량,23시_시간당백색광량,23시_시간당적색광량,23시_시간당청색광량,day
0,25.300000,81.835000,536.016667,1.407439,0.00,0.0,0.0,0.0,25.680357,81.264286,...,0.0,25.873333,81.900001,517.933333,1.388323,0.00,0.0,0.0,0.0,0
1,25.806666,81.765000,523.216667,1.390210,126.00,0.0,0.0,0.0,25.981034,81.329310,...,0.0,26.196667,81.285000,513.366667,1.384234,0.00,0.0,0.0,0.0,1
2,26.098334,81.778334,516.416667,1.385905,119.57,0.0,0.0,0.0,26.313559,81.269491,...,0.0,25.491667,81.186667,524.866667,1.395641,126.00,0.0,0.0,0.0,2
3,25.438333,81.058333,531.450000,1.396743,126.00,0.0,0.0,0.0,25.783051,80.571186,...,0.0,25.145001,81.423334,791.500000,1.403352,0.00,0.0,0.0,0.0,3
4,25.470000,81.300001,543.600000,1.409774,0.00,0.0,0.0,0.0,25.462712,81.476272,...,0.0,25.278334,82.026666,538.433333,1.407541,119.57,0.0,0.0,0.0,4


In [255]:
train_y.head()

,0
0,0.167719
1,0.181787
2,0.265921
3,0.423650
4,0.475272


In [256]:
# 데이터 스케일링
p_scaler = StandardScaler()
p_scaler.fit(train_X)
train_scaled = p_scaler.transform(train_X)
test_scaled = p_scaler.transform(test_X)
train_y = modified_target(train_y)

# Modeling
- LSTM : 순환 신경망(RNN)의 일종으로, 시간의 흐름에 따라 변하는 시계열 데이터를 잘 다룰 수 있는 모델
- XGBoost : 결정 트리 기반의 앙상블 학습 모델
- LightGBM : XGBoost와 유사한 앙상블 학습 기법이지만, 더 빠르고 효율적인 학습을 위해 최적화된 모델
- SegRNN : 시계열 데이터의 특정 구간을 분할하여 각 구간의 특징을 학습하는 모델
- Random Forest Regression : 여러 개의 결정 트리를 사용하여 예측을 수행하는 앙상블 학습 모델

참고

https://paperswithcode.com/area/time-series (모델 성능 참고)

https://towardsdatascience.com/ensemble-models-5a62d4f4cb0c (앙상블 모델 설명)

## 1. LSTM (Long Short-Term Memory)
LSTM은 장기 의존성 문제를 해결하는 순환 신경망

장점:
- 시계열 데이터의 장기 패턴을 잘 학습함
- 과거 데이터의 중요한 정보를 오래 유지

단점:
- 학습 시간이 길어질 수 있음
- 많은 하이퍼파라미터로 인해 튜닝이 복잡할 수 있음

In [119]:
def create_lstm_model(input_shape):
    model = Sequential()

    model.add(LSTM(512, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(256, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(LSTM(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    return model

In [152]:
def preprocess_data(X, time_steps):
    num_samples, num_features = X.shape
    num_samples = (num_samples // time_steps) * time_steps  # 나머지 샘플 제거
    X = X[:num_samples].reshape(-1, time_steps, num_features)
    return X

In [153]:
train_np_X = np.array(train_X)
train_np_y = np.array(train_y)
test_np_X = np.array(test_X)
test_np_y = np.array(test_y)

time_steps = 24

train_np_X = preprocess_data(train_np_X, time_steps)
test_np_X = preprocess_data(test_np_X, time_steps)

In [123]:
def lstm_cv(train_X, train_y, cv_folds=7, epochs=50, batch_size=32, time_steps=10):
    start_time = time.time()

    train_X = train_X[:num_samples].reshape(-1, time_steps, num_features)

    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    rmse_scores = []
    histories = []

    for train_index, val_index in kf.split(train_X):
        X_train, X_val = train_X[train_index], train_X[val_index]
        y_train, y_val = train_y[train_index], train_y[val_index]

        model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                            validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

        histories.append(history.history)

        y_pred = model.predict(X_val)
        y_pred = y_pred.flatten()

        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        rmse_scores.append(rmse)

    mean_rmse = np.mean(rmse_scores)
    elapsed_time = time.time() - start_time
    print(f"Time: {elapsed_time:.4f} seconds")

    return round(mean_rmse, 3)

In [124]:
rmse_score = lstm_cv(train_np_X, train_np_y, time_steps=time_steps)
print(f"Average RMSE: {rmse_score}")

1/1 [==============================] - 0s 482ms/step
Time: 182.6565 seconds
Average RMSE: 4.976


## 2. XGBoost (Extreme Gradient Boosting)
XGBoost는 결정 트리 기반의 강력한 앙상블 학습 기법

장점:
- 빠른 연산 속도와 높은 예측 성능을 제공
- 과적합 방지를 위한 다양한 규제 기법을 포함

단점:
- 큰 데이터셋에서는 메모리 사용량이 많다
- 하이퍼파라미터 튜닝이 복잡할 수 있다

In [77]:
# 모델 정의
def xgb_cv(train_X, train_y, cv_folds=7, n_estimators=100, random_state=42, n_jobs=-1):
    """
    Parameters:
    - cv_folds: 데이터를 7개의 그룹으로 나누어 교차 검증을 수행
    - n_estimators: 100개의 의사결정 트리를 사용하여 모델을 학습
    - n_jobs: 병렬로 실행할 작업 수. 작업할 컴퓨터의 코어
    """
    start_time = time.time()

    xgb = xgboost.XGBRegressor(n_estimators=n_estimators,
                               tree_method='exact',
                               random_state=random_state)

    cv_results = cross_validate(xgb, train_X, train_y,
                                scoring='neg_mean_squared_error',
                                return_train_score=True,
                                cv=cv_folds, n_jobs=n_jobs)

    test_rmse_scores = np.sqrt(-cv_results['test_score'])
    mean_rmse = np.mean(test_rmse_scores)

    elapsed_time = time.time() - start_time
    print(f"Time: {elapsed_time:.4f} seconds")

    return round(mean_rmse, 3)

In [ ]:
rmse_score = xgb_cv(train_X, train_y)
print(f"Average RMSE: {rmse_score}")

Time: 15.0750 seconds
Average RMSE: 3.322


In [ ]:
# xgb 모델 저장
xgb = xgboost.XGBRegressor(n_estimators=100,
                            tree_method='exact',
                            random_state=42).fit(train_scaled,train_y)


pickle.dump(xgb,open('./xgboost_best','wb'))

In [ ]:
# 저장한 xgb 모델 불러오기
xgb_model = pickle.load(open('./xgboost_best', 'rb'))

In [ ]:
# 수정했던 타겟 값 원래대로 되돌리기
def revert_target(pred_list):
    result = []
    for i in range(len(pred_list)):
        if i % 28 == 0:
            result.append(pred_list[i])
        else:
            num = result[-1] + pred_list[i]
            result.append(num)
    return result

# 일별 상추 중량 확인
def day_result(pred_list):
    result = pd.DataFrame(columns=list(range(28)))
    for idx in range(5):
        day_data = pd.Series(pred_list[28 * idx:28 * (idx + 1)])
        result = pd.concat([result, pd.DataFrame([day_data.values], columns=result.columns)], ignore_index=True)
    return result

In [ ]:
# 예측
xgb_pred= xgb.predict(test_scaled)
xgb_pred = revert_target(xgb_pred)

# 케이스별 데이터 확인
dr = day_result(xgb_pred)
dr

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.759374,1.399643,1.867373,2.249509,3.011195,3.581326,4.262211,5.203165,6.535791,7.912818,...,42.534607,45.704304,52.672302,61.386917,70.666550,80.928986,87.522446,93.496506,99.536690,110.498131
1,0.248680,0.477684,0.760966,1.003431,1.377403,1.895852,2.253497,2.662163,3.300667,4.086874,...,25.668764,29.471550,35.315529,39.398315,44.499340,51.329929,57.292030,62.249161,68.711594,75.677383
2,0.751162,1.145554,1.656573,2.051573,2.469525,2.947980,3.555903,4.365896,5.243775,5.759140,...,18.800224,21.219551,23.080130,25.448927,28.688431,31.014809,32.470478,33.619125,34.472973,36.642677
3,0.413389,1.028885,1.427417,1.777115,2.795958,3.384964,4.284807,5.526300,7.678939,8.570216,...,36.760750,41.048347,47.562801,56.167881,63.953716,75.812515,82.722076,90.674820,97.481117,108.084869
4,0.555707,1.311993,1.909429,2.875903,3.824496,4.810809,5.651744,6.511152,8.133231,9.764557,...,30.980816,33.537544,38.656422,42.810745,47.253296,51.470730,57.735634,66.712341,69.752022,76.751907


## 3. LightGBM (Light Gradient Boosting Machine)
LightGBM은 대용량 데이터셋에서 빠르고 효율적인 학습을 위한 앙상블 기법

장점:
- 학습 속도가 매우 빠름
- 메모리 효율성이 높음

단점:

- 작은 데이터셋에서는 성능이 저하될 수 있음
- 데이터 전처리와 형식에 민감할 수 있음

In [168]:
def lgb_cv(train_X, train_y, cv_folds=7, n_estimators=100, random_state=42, n_jobs=-1):
    start_time = time.time()

    lgbm = lgb.LGBMRegressor(n_estimators=n_estimators,
                           random_state=random_state)

    cv_results = cross_validate(lgbm, train_X, train_y,
                               scoring='neg_mean_squared_error',
                               return_train_score=True,
                               cv=cv_folds, n_jobs=n_jobs)

    test_rmse_scores = np.sqrt(-cv_results['test_score'])
    mean_rmse = np.mean(test_rmse_scores)

    elapsed_time = time.time() - start_time
    print(f"Time: {elapsed_time:.4f} seconds")

    return round(mean_rmse, 3)

In [169]:
rmse_score = lgb_cv(train_X, train_y)
print(f"Average RMSE: {rmse_score}")

Time: 12.0590 seconds
Average RMSE: 3.11


##4. SegRNN (Segmental Recurrent Neural Network)

SegRNN은 시계열 데이터를 특정 구간으로 나누어 학습하는 순환 신경망

장점:

- 구간별 특징을 잘 반영하여 예측 정확도가 높다
- 특정 이벤트나 패턴을 더 잘 캡처할 수 있다

단점:

- 데이터 구간 설정이 어려울 수 있다
- 계산 복잡도가 높아질 수 있다

## 5. Random Forest Regression
여러 개의 결정 트리를 사용하여 예측을 수행하는 앙상블 학습 모델

장점:

- 여러 트리의 예측을 평균화하기 때문에 과적합 가능성이 낮다
- 구현이 비교적 간단하며, 기본 설정으로도 좋은 성능을 보임

단점:
- 많은 트리를 생성하므로 메모리 사용량이 많을 수 있음
- 많은 트리를 거쳐 예측하기 때문에 실시간 예측에는 부적합할 수 있음

In [257]:
def rf_cv(train_X, train_y, cv_folds=7, n_estimators=100, random_state=42, n_jobs=-1):
    start_time = time.time()

    rf = RandomForestRegressor(n_estimators=n_estimators,
                               random_state=random_state,
                               n_jobs=n_jobs)

    cv_results = cross_validate(rf, train_X, train_y,
                                scoring='neg_mean_squared_error',
                                return_train_score=True,
                                cv=cv_folds, n_jobs=n_jobs)

    test_rmse_scores = np.sqrt(-cv_results['test_score'])
    mean_rmse = np.mean(test_rmse_scores)

    elapsed_time = time.time() - start_time
    print(f"Time: {elapsed_time:.4f} seconds")

    return round(mean_rmse, 3)

In [258]:
rmse_score = rf_cv(train_X, train_y)
print(f"Average RMSE: {rmse_score}")

Time: 52.0971 seconds
Average RMSE: 3.027


# BenchMark